In [74]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
data = pd.read_excel('/content/AJGT.xlsx')
data2 = pd.read_csv("/content/DATA_SET_CP.csv")

In [29]:
data2.head()

,ممتازة بصراحه مريحة خاصتا في الخرجات الي من غير عربية,1
0,عملي,1
1,جيد نوع,0
2,اشتريت السلعه ودفعت ثمنها من الماستر كارد وتم ...,-1
3,شنطة جميلة وأنيقة وسعرها مناسب جدا,-1
4,حلوه,1


In [30]:
data.head()

,ID,Feed,Sentiment
0,1,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,Positive
1,2,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,Negative
2,3,كله رائع بجد ربنا يكرمك,Positive
3,4,لسانك قذر يا قمامه,Negative
4,5,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,Negative


In [31]:
columns = ['Feed','Sentiment']
data2.columns = columns

In [32]:
data2 = data2[data2['Sentiment'] != 0]


In [33]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22401 entries, 0 to 24858
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Feed       22401 non-null  object
 1   Sentiment  22401 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 525.0+ KB


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         1800 non-null   int64 
 1   Feed       1800 non-null   object
 2   Sentiment  1800 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.3+ KB


In [35]:
data.drop(columns=['ID'], axis=1, inplace=True)

In [36]:
data

,Feed,Sentiment
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,Positive
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,Negative
2,كله رائع بجد ربنا يكرمك,Positive
3,لسانك قذر يا قمامه,Negative
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,Negative
...,...,...
1795,يلعن شرف خواتك,Negative
1796,يلعنك نذل وكلب,Negative
1797,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,Positive
1798,ينور عليك ويكثر من امثالك,Positive


In [37]:
mapping = {'Positive':1, 'Negative':-1}
data['Sentiment']=data['Sentiment'].map(mapping)

In [13]:
data

,Feed,Sentiment
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,1
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,0
2,كله رائع بجد ربنا يكرمك,1
3,لسانك قذر يا قمامه,0
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,0
...,...,...
1795,يلعن شرف خواتك,0
1796,يلعنك نذل وكلب,0
1797,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,1
1798,ينور عليك ويكثر من امثالك,1


In [38]:
df = pd.concat([data, data2])

In [39]:
df

,Feed,Sentiment
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,1
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,-1
2,كله رائع بجد ربنا يكرمك,1
3,لسانك قذر يا قمامه,-1
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,-1
...,...,...
24854,تضامنا مع باسم يوسف,1
24855,حد واخد باله ان القنوات الدينيه كلها سايبة الا...,1
24856,حضرتك بتحب منير وانغام وكمان حلمي ومكي وبتعشق ...,1
24857,حيغطي علي باسم يوسف هههههههههه بيضحكني اكتر من...,1


In [72]:
df['Sentiment'].value_counts()

,count
Sentiment,
1,17287
-1,6914


In [44]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
stop_words = stopwords.words()
arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


In [45]:
def preprocess(text):

    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    text = re.sub(arabic_diacritics, '', text)
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

In [46]:
df['Feed'] = df['Feed'].apply(preprocess)

In [48]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['Feed'])

In [75]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, df['Sentiment'])

In [76]:
x_train,x_test,y_train,y_test = train_test_split(X_resampled ,y_resampled, test_size=0.2, random_state=42)

In [91]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)

DecisionTreeClassifier()

In [78]:
y_test_pred = classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8932754880694144


In [92]:
joblib.dump(classifier, 'model.pkl')

['model.pkl']

In [88]:
classifier = joblib.load('model.pkl')

In [90]:
text = "انا تعبان"
text = preprocess(text)
text = tfidf.transform([text])
prediction = classifier.predict(text)
if prediction == 1:
  print("Positive")
else:
  print("Negative")

Negative
